In [1]:
# Cell 1: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import mean_squared_error, accuracy_score
from tqdm import tqdm

# Cell 2: Custom Dataset Class
class StudentFeedbackDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file).sample(frac=1, random_state=42)
        self.comments = self.data['comments']
        
        self.labels_star = self.data[['student_star']]
        self.labels_difficult = self.data[['student_difficult']]
        self.labels_feedback = self.data[['gives_good_feedback']]
        self.labels_caring = self.data[['caring']]
        self.labels_respected = self.data[['respected']]
        self.tokenizer = AutoTokenizer.from_pretrained('roberta-base')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        comment = self.comments.iloc[idx]
        encoded_comment = self.tokenizer(comment, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        
        labels_star = self.labels_star.iloc[idx].values.astype('float32')
        labels_difficult = self.labels_difficult.iloc[idx].values.astype('float32')
        labels_feedback = self.labels_feedback.iloc[idx].values.astype('float32')
        labels_caring = self.labels_caring.iloc[idx].values.astype('float32')
        labels_respected = self.labels_respected.iloc[idx].values.astype('float32')
        # Squeeze the batch dimension from encoded_comment tensors
        encoded_comment = {key: value.squeeze(0) for key, value in encoded_comment.items()}
        return encoded_comment, torch.tensor(labels_star), torch.tensor(labels_difficult), torch.tensor(labels_feedback), torch.tensor(labels_caring), torch.tensor(labels_respected)

# Cell 3: Neural Network Model
class StudentFeedbackModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(StudentFeedbackModel, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
        self.fc1 = nn.Linear(768, 512)  # 768 from RoBERTa output + additional features
        self.fc2 = nn.Linear(512, 256)  # Additional hidden layer
        self.fc3 = nn.Linear(256, 64)  # Another hidden layer
        self.fc4 = nn.Linear(64, 64)  # Another hidden layer
        self.fc2_star = nn.Linear(64, 1)  # Output for student_star
        self.fc2_difficult = nn.Linear(64, 1)  # Output for student_difficult
        self.fc2_feedback = nn.Linear(64, 1)  # Output for gives_good_feedback
        self.fc2_caring = nn.Linear(64, 1)  # Output for caring
        self.fc2_respected = nn.Linear(64, 1)  # Output for respected
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, encoded_comment):
        roberta_output = self.roberta(**encoded_comment).last_hidden_state[:, 0, :]
        x = roberta_output
        x = self.relu(self.fc1(x))
        # x = self.dropout(x)
        x = self.relu(self.fc2(x))
        # x = self.dropout(x)
        x = self.relu(self.fc3(x))
        # x = self.dropout(x)
        labels_xtra_layer = self.relu(self.fc4(x))
        star_outputs = self.fc2_star(x)
        difficult_outputs = self.fc2_difficult(x)
        feedback_output = self.sigmoid(self.fc2_feedback(labels_xtra_layer))
        caring_output = self.sigmoid(self.fc2_caring(labels_xtra_layer))
        respected_output = self.sigmoid(self.fc2_respected(labels_xtra_layer))
        return star_outputs, difficult_outputs, feedback_output, caring_output, respected_output


In [2]:
# Cell 4: Training Function
def train_model(model, train_dataloader, test_dataloader, criterion_star, criterion_prob, optimizer, epochs=10, evaluation_interval=1, patience=3):
    model.train()
    model.to('cuda:1')
    best_loss = float('inf')
    patience_counter = 0
    for epoch in range(epochs):
        running_loss = 0.0
        for encoded_comment, labels_star, labels_difficult, labels_feedback, labels_caring, labels_respected in tqdm(train_dataloader):
            optimizer.zero_grad()
            
            # Move inputs and labels to GPU
            encoded_comment = {key: value.to('cuda:1') for key, value in encoded_comment.items()}
            labels_star = labels_star.to('cuda:1')
            labels_difficult = labels_difficult.to('cuda:1')
            labels_feedback = labels_feedback.to('cuda:1')
            labels_caring = labels_caring.to('cuda:1')
            labels_respected = labels_respected.to('cuda:1')
            
            # Forward pass
            star_outputs, difficult_outputs, feedback_output, caring_output, respected_output = model(encoded_comment)
            
            # Calculate losses
            loss_star = criterion_star(star_outputs, labels_star)
            loss_difficult = criterion_star(difficult_outputs, labels_difficult)
            loss_feedback = criterion_prob(feedback_output, labels_feedback)
            loss_caring = criterion_prob(caring_output, labels_caring)
            loss_respected = criterion_prob(respected_output, labels_respected)
            loss = 1.0 * loss_star + 0.5 * loss_difficult + 0.2 * (loss_feedback + loss_caring + loss_respected)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}')
        
        # Early stopping
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping triggered.')
                break
        
        # Evaluate the model at the specified interval
        if (epoch + 1) % evaluation_interval == 0:
            evaluate_model(model, test_dataloader)

In [3]:
# Cell 5: Evaluation Function
def evaluate_model(model, dataloader):
    model.eval()
    model.to('cuda:1')
    star_preds = []
    star_labels = []
    difficult_preds = []
    difficult_labels = []
    feedback_preds = []
    feedback_labels = []
    caring_preds = []
    caring_labels = []
    respected_preds = []
    respected_labels = []
    with torch.no_grad():
        for encoded_comment, labels_star, labels_difficult, labels_feedback, labels_caring, labels_respected in dataloader:
            # Move inputs and labels to GPU
            encoded_comment = {key: value.to('cuda:1') for key, value in encoded_comment.items()}
            labels_star = labels_star.to('cuda:1')
            labels_difficult = labels_difficult.to('cuda:1')
            labels_feedback = labels_feedback.to('cuda:1')
            labels_caring = labels_caring.to('cuda:1')
            labels_respected = labels_respected.to('cuda:1')
            
            # Forward pass
            star_outputs, difficult_outputs, feedback_output, caring_output, respected_output = model(encoded_comment)
            
            # Collect predictions and true labels
            star_preds.append(star_outputs.cpu())
            star_labels.append(labels_star.cpu())
            difficult_preds.append(difficult_outputs.cpu())
            difficult_labels.append(labels_difficult.cpu())
            feedback_preds.append(feedback_output.cpu())
            feedback_labels.append(labels_feedback.cpu())
            caring_preds.append(caring_output.cpu())
            caring_labels.append(labels_caring.cpu())
            respected_preds.append(respected_output.cpu())
            respected_labels.append(labels_respected.cpu())
    
    # Convert lists to tensors
    star_preds = torch.cat(star_preds, dim=0).numpy()
    star_labels = torch.cat(star_labels, dim=0).numpy()
    difficult_preds = torch.cat(difficult_preds, dim=0).numpy()
    difficult_labels = torch.cat(difficult_labels, dim=0).numpy()
    feedback_preds = torch.cat(feedback_preds, dim=0).numpy()
    feedback_labels = torch.cat(feedback_labels, dim=0).numpy()
    caring_preds = torch.cat(caring_preds, dim=0).numpy()
    caring_labels = torch.cat(caring_labels, dim=0).numpy()
    respected_preds = torch.cat(respected_preds, dim=0).numpy()
    respected_labels = torch.cat(respected_labels, dim=0).numpy()
    
    # Calculate evaluation metrics
    star_mse = mean_squared_error(star_labels, star_preds)
    difficult_mse = mean_squared_error(difficult_labels, difficult_preds)
    feedback_accuracy = accuracy_score(feedback_labels, (feedback_preds > 0.5).astype(int))
    caring_accuracy = accuracy_score(caring_labels, (caring_preds > 0.5).astype(int))
    respected_accuracy = accuracy_score(respected_labels, (respected_preds > 0.5).astype(int))
    
    print(f'Mean Squared Error for star predictions: {star_mse:.8f}')
    print(f'Mean Squared Error for difficult predictions: {difficult_mse:.8f}')
    print(f'Accuracy for feedback predictions: {feedback_accuracy:.8f}')
    print(f'Accuracy for caring predictions: {caring_accuracy:.8f}')
    print(f'Accuracy for respected predictions: {respected_accuracy:.8f}')

# Cell 6: Main Execution
if __name__ == "__main__":
    # Load and preprocess the data
    dataset = StudentFeedbackDataset('data.csv')
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


    # Model parameters
    input_dim = 0
    hidden_dim = 64
    model = StudentFeedbackModel(input_dim, hidden_dim)

    # Loss and optimizer
    criterion_star = nn.MSELoss()
    criterion_prob = nn.BCELoss()  # Binary Cross Entropy Loss for probability labels
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

    # Train the model
    train_model(model, train_dataloader, test_dataloader, criterion_star, criterion_prob, optimizer, epochs=20, evaluation_interval=1)

    # Save the trained model
    torch.save(model.state_dict(), 'student_feedback_model.pth')



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 238/238 [00:46<00:00,  5.17it/s]


Epoch [1/20], Loss: 3.8281
Mean Squared Error for star predictions: 0.97528005
Mean Squared Error for difficult predictions: 1.43386257
Accuracy for feedback predictions: 0.69599579
Accuracy for caring predictions: 0.73234984
Accuracy for respected predictions: 0.72049526


  8%|▊         | 20/238 [00:03<00:43,  5.05it/s]


KeyboardInterrupt: 